# Quantum Convolutional Neural Network

This is an implementation of 8 Qubit QCNN circuit testing various ansatze.

In [1]:
import pennylane as qml
from pennylane import numpy as np
np.random.seed(0)

# QCNN Circuit

Here are various unitary ansatze to test

In [21]:
# Unitraies for Convolutional Layers 
def U_TTN(params, wires): # 2 params
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[1], wires = wires[1])
    qml.CNOT(wires = [wires[0], wires[1]])

def U_5(params, wires): # 10 params
    qml.RX(params[0], wires = wires[0])
    qml.RX(params[1], wires = wires[1])
    qml.RZ(params[2], wires = wires[0])
    qml.RZ(params[3], wires = wires[1])
    qml.CRZ(params[4], wires = [wires[1], wires[0]])
    qml.CRZ(params[5], wires = [wires[0], wires[1]])
    qml.RX(params[6], wires = wires[0])
    qml.RX(params[7], wires = wires[1])
    qml.RZ(params[8], wires = wires[0])
    qml.RZ(params[9], wires = wires[1])
    
def U_6(params, wires): # 10 params
    qml.RX(params[0], wires = wires[0])
    qml.RX(params[1], wires = wires[1])
    qml.RZ(params[2], wires = wires[0])
    qml.RZ(params[3], wires = wires[1])
    qml.CRX(params[4], wires = [wires[1], wires[0]])
    qml.CRX(params[5], wires = [wires[0], wires[1]])
    qml.RX(params[6], wires = wires[0])
    qml.RX(params[7], wires = wires[1])
    qml.RZ(params[8], wires = wires[0])
    qml.RZ(params[9], wires = wires[1])

def U_9(params, wires): # 2 params
    qml.Hadamard(wires = wires[0])
    qml.Hadamard(wires = wires[1])
    qml.CZ(wires = [wires[0],wires[1]])
    qml.RX(params[0], wires = wires[0])
    qml.RX(params[1], wires = wires[1])

def U_13(params, wires): # 6 params
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[1], wires = wires[1])
    qml.CRZ(params[2], wires = [wires[1], wires[0]])
    qml.RY(params[3], wires = wires[0])
    qml.RY(params[4], wires = wires[1])
    qml.CRZ(params[5], wires = [wires[0], wires[1]])

def U_14(params, wires): # 6 params
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[1], wires = wires[1])
    qml.CRX(params[2], wires = [wires[1], wires[0]])
    qml.RY(params[3], wires = wires[0])
    qml.RY(params[4], wires = wires[1])
    qml.CRX(params[5], wires = [wires[0], wires[1]])

def U_15(params, wires): # 4 params
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[1], wires = wires[1])
    qml.CNOT(wires = [wires[1], wires[0]])
    qml.RY(params[2], wires = wires[0])
    qml.RY(params[3], wires = wires[1])
    qml.CNOT(wires = [wires[0], wires[1]])

# Unitraies for Pooling and Fully Connected Layers
def V_0(theta, wires):
    qml.CRZ(theta, wires = [wires[0], wires[1]])

def V_1(theta, wires):
    qml.CRX(theta, wires = [wires[0], wires[1]])

def F(theta, wires):
    qml.CRZ(theta, wires = [wires[0], wires[1]])

Define general layers that will be used in the circuits.

In [3]:
# Convolution Layer1
def conv_layer1(U, params):
    U(params, wires = [0,7])
    for i in range (0,8,2):
        U(params, wires = [i, i+1])
    for i in range (1,7,2):
        U(params, wires = [i, i+1])
    
def conv_layer2(U, params):
    U(params, wires = [0,2])
    U(params, wires = [4,6])
    U(params, wires = [2,4])
    U(params, wires = [0,6])
    
def pooling_layer1(V_0, V_1, params):
    for i in range(0,8,2):
        V_0(params[0], wires = [i+1, i])
    for i in range(0,8,2):
        qml.PauliX(wires = i+1)
    for i in range(0,8,2):
        V_1(params[1], wires = [i+1, i])
        

def pooling_layer2(V_0, V_1, params): # 2params
    V_0(params[0], wires = [2,0])
    V_0(params[0], wires = [6,4])
    
    qml.PauliX(wires = 2)
    qml.PauliX(wires = 6)
    
    V_1(params[1], wires = [2,0])
    V_1(params[1], wires = [6,4])

Here we define various possible embedding methods.

In [4]:
from pennylane.templates.embeddings import AmplitudeEmbedding, AngleEmbedding

def data_embedding(X, embedding_type = 'Amplitude'):
    if embedding_type == 'Amplitude':
        AmplitudeEmbedding(X, wires = range(8), normalize = True)
    elif embedding_type == 'Angle':
        AngleEmbedding(X, wires = range(8), rotation = 'Y')

Define QCNN circuit with given Unitary Ansatz and embedding method.

In [5]:
dev = qml.device('default.qubit', wires = 8)

@qml.qnode(dev)
def QCNN(X, params, U, U_params, embedding_type = 'Amplitude'):

    param1 = params[0:U_params]
    param2 = params[U_params:U_params + 2]
    param3 = params[U_params + 2: 2*U_params + 2]
    param4 = params[2*U_params + 2: 2*U_params + 4]
    param5 = params[2*U_params + 4]
    
    # Data Embedding
    data_embedding(X, embedding_type = embedding_type)
    
    #Quantum Convolutional Neural Network
    if U == 'U_TTN':
        conv_layer1(U_TTN, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_TTN, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    elif U == 'U_5':
        conv_layer1(U_5, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_5, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    elif U == 'U_6':
        conv_layer1(U_6, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_6, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    elif U == 'U_9':
        conv_layer1(U_9, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_9, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    elif U == 'U_13':
        conv_layer1(U_13, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_13, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    elif U == 'U_14':
        conv_layer1(U_14, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_14, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    elif U == 'U_15':
        conv_layer1(U_15, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_15, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    return qml.expval(qml.PauliZ(4))

# Training Quantum Circuits

In [34]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

def cost(params, X, Y, U, U_params, embedding_type):
    predictions = [QCNN(x, params, U, U_params, embedding_type) for x in X]
    return square_loss(Y, predictions)

def accuracy_test(predictions, labels):
    acc = 0
    for l,p in zip(labels, predictions):
        if np.abs(l - p) < 1:
            acc = acc + 1
    return acc / len(labels)

In [23]:
def circuit_training(X_train, Y_train, U, U_params, embedding_type):
    total_params = U_params * 2 + 2 * 2 + 1
    params = np.random.randn(total_params)
    steps = 100
    learning_rate = 0.1
    batch_size =25
    opt = qml.NesterovMomentumOptimizer(learning_rate)
    
    for it in range(steps):
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        X_batch = [X_train[i] for i in batch_index]
        Y_batch = [Y_train[i] for i in batch_index]
        params, cost_new = opt.step_and_cost(lambda v: cost(v, X_batch, Y_batch, U, U_params, embedding_type), params)
        if it % 10 == 0:
            print("iteration: ", it, " cost: ", cost_new)
    return params

### MNIST Data loading and processing

In [8]:
import tensorflow as tf
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses

def data_load_and_process(classes = [0,1], feature_reduction = 'resize256'):
    
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0 #normalize the data
    
    x_train_filter_01 = np.where((y_train == classes[0]) | (y_train == classes[1]))
    x_test_filter_01 = np.where((y_test == classes[0]) | (y_test == classes[1]))
    
    x_train_01, x_test_01 = x_train[x_train_filter_01], x_test[x_test_filter_01]
    y_train_01, y_test_01 = y_train[x_train_filter_01], y_test[x_test_filter_01]
    
    y_train_01 = [1 if y ==classes[0] else -1 for y in y_train_01]
    y_test_01 = [1 if y ==classes[1] else -1 for y in y_test_01]
    
    if feature_reduction == 'resize256':   
        x_train_01 = tf.image.resize(x_train_01[:], (256, 1)).numpy()
        x_test_01 = tf.image.resize(x_test_01[:], (256, 1)).numpy()
        x_train_01, x_test_01 = tf.squeeze(x_train_01), tf.squeeze(x_test_01) 
        return x_train_01, x_test_01, y_train_01, y_test_01
    
    elif feature_reduction == 'pca8':
        x_train_01 = tf.image.resize(x_train_01[:], (784, 1)).numpy()
        x_test_01 = tf.image.resize(x_test_01[:], (784, 1)).numpy()
        x_train_01, x_test_01 = tf.squeeze(x_train_01), tf.squeeze(x_test_01)
        
        pca = PCA(8)
        x_train_01 = pca.fit_transform(x_train_01)
        x_test_01 = pca.transform(x_test_01) 
        return x_train_01, x_test_01, y_train_01, y_test_01

    elif feature_reduction == 'autoencoder8':
        latent_dim = 8 
        class Autoencoder(Model):
            def __init__(self, latent_dim):
                super(Autoencoder, self).__init__()
                self.latent_dim = latent_dim   
                self.encoder = tf.keras.Sequential([
                layers.Flatten(),
                  layers.Dense(latent_dim, activation='relu'),
                ])
                self.decoder = tf.keras.Sequential([
                layers.Dense(784, activation='sigmoid'),
                layers.Reshape((28, 28))
                ])
            def call(self, x):
                encoded = self.encoder(x)
                decoded = self.decoder(encoded)
                return decoded

        autoencoder = Autoencoder(latent_dim)
        
        autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())
        autoencoder.fit(x_train_01, x_train_01,
                epochs=10,
                shuffle=True,
                validation_data=(x_test_01, x_test_01))
        
        x_train_01, x_test_01 = autoencoder.encoder(x_train_01).numpy(), autoencoder.encoder(x_test_01).numpy()
        return x_train_01, x_test_01, y_train_01, y_test_01


Use PCA and Autoencoder to reduce it into 8 features.

# Results

In [17]:
Unitaries = ['U_TTN', 'U_5', 'U_6', 'U_9', 'U_13', 'U_14', 'U_15']
U_num_params = [2, 10, 10, 4, 6, 6, 4]
Encodings = ['resize256', 'pca8', 'autoencoder8']

In [18]:
x_train_resize256, x_test_resize256, y_train_resize256, y_test_resize256 = data_load_and_process(classes = [0,1], feature_reduction = 'resize256')
x_train_pca8, x_test_pca8, y_train_pca8, y_test_pca8 = data_load_and_process(classes = [0,1], feature_reduction = 'pca8')
x_train_autoencoder8, x_test_autoencoder8, y_train_autoencoder8, y_test_autoencoder8 = data_load_and_process(classes = [0,1], feature_reduction = 'autoencoder8')

Train on 12665 samples, validate on 2115 samples
Epoch 1/10
12665/12665 [==============================] - 1s 62us/sample - loss: 0.0640 - val_loss: 0.0346
Epoch 2/10
12665/12665 [==============================] - 1s 41us/sample - loss: 0.0300 - val_loss: 0.0272
Epoch 3/10
12665/12665 [==============================] - 1s 40us/sample - loss: 0.0260 - val_loss: 0.0242
Epoch 4/10
12665/12665 [==============================] - 1s 41us/sample - loss: 0.0232 - val_loss: 0.0220
Epoch 5/10
12665/12665 [==============================] - 1s 40us/sample - loss: 0.0217 - val_loss: 0.0209
Epoch 6/10
12665/12665 [==============================] - 1s 40us/sample - loss: 0.0209 - val_loss: 0.0203
Epoch 7/10
12665/12665 [==============================] - 1s 48us/sample - loss: 0.0203 - val_loss: 0.0197
Epoch 8/10
12665/12665 [==============================] - 1s 46us/sample - loss: 0.0198 - val_loss: 0.0193
Epoch 9/10
12665/12665 [==============================] - 1s 41us/sample - loss: 0.0194 - val_l


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [35]:
I = len(Unitaries)
J = len(Encodings)
All_predictions = []

for i in range(I):
    for j in range(J):
        U = Unitaries[i]
        U_params = U_num_params[i]
        Encoding = Encodings[j]
        if Encoding == 'resize256':
            Embedding = 'Amplitude'
            X_train, X_test, Y_train, Y_test = x_train_resize256, x_test_resize256, y_train_resize256, y_test_resize256
        elif Encoding == 'pca8':
            Embedding = 'Angle'
            X_train, X_test, Y_train, Y_test = x_train_pca8, x_test_pca8, y_train_pca8, y_test_pca8
        elif Encoding == 'autoencoder8':
            Embedding = 'Angle'
            X_train, X_test, Y_train, Y_test = x_train_autoencoder8, x_test_autoencoder8, y_train_autoencoder8, y_test_autoencoder8
        
        print("\n")
        print("Loss History for " + U + " " + Encoding)
        trained_params = circuit_training(X_train, Y_train, U, U_params, Embedding)

        predictions = [QCNN(x, trained_params, U, U_params, Embedding) for x in X_test]
        All_predictions.append(predictions)
        accuracy = accuracy_test(predictions, Y_test)
        print("Accuracy for " + U + " " + Encoding + " :" + str(accuracy))



Loss History for U_TTN resize256
iteration:  0  cost:  1.0942672649744019
iteration:  10  cost:  0.8531428899961425
iteration:  20  cost:  0.7873782303591627
iteration:  30  cost:  0.7461528416652592
iteration:  40  cost:  0.6980639428506551
iteration:  50  cost:  0.6975354376885243
iteration:  60  cost:  0.727635809626079
iteration:  70  cost:  0.7030198372359991
iteration:  80  cost:  0.6423794458980574
iteration:  90  cost:  0.6891742354525557
Accuracy for U_TTN resize256 :0.15555555555555556


Loss History for U_TTN pca8
iteration:  0  cost:  0.9746653475415564
iteration:  10  cost:  1.046105677817918
iteration:  20  cost:  1.027484537498441
iteration:  30  cost:  1.0159608709191443
iteration:  40  cost:  0.9638455832074588
iteration:  50  cost:  0.9188259550799254
iteration:  60  cost:  0.9871963402266046
iteration:  70  cost:  0.969281733018343
iteration:  80  cost:  0.9271242124286451
iteration:  90  cost:  0.964488033443634
Accuracy for U_TTN pca8 :0.40709219858156026


Loss 

iteration:  50  cost:  0.8941169959995452
iteration:  60  cost:  0.9131950857114964
iteration:  70  cost:  0.6347706463054172
iteration:  80  cost:  1.0025654080905404
iteration:  90  cost:  0.7496178193503394
Accuracy for U_14 pca8 :0.32907801418439714


Loss History for U_14 autoencoder8
iteration:  0  cost:  1.5964540766920303
iteration:  10  cost:  1.1958022408925195
iteration:  20  cost:  1.137576306851333
iteration:  30  cost:  1.103143024504743
iteration:  40  cost:  0.9044117193541353
iteration:  50  cost:  0.9403079976271786
iteration:  60  cost:  0.9385615930162322
iteration:  70  cost:  0.8417815234948374
iteration:  80  cost:  0.7891746881478877
iteration:  90  cost:  0.982912851848127
Accuracy for U_14 autoencoder8 :0.3177304964539007


Loss History for U_15 resize256
iteration:  0  cost:  0.6214040636756165
iteration:  10  cost:  0.6165501428963386
iteration:  20  cost:  0.544737023102904
iteration:  30  cost:  0.41421216408930767
iteration:  40  cost:  0.4417434550104557